In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from numpy.random import default_rng
from soap import reshape_soaps, compute_soap_density
from ase.data import chemical_symbols
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import rascaline, sys, re, itertools
from metatensor import Labels
import ase.io as aseio
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from matplotlib.backends.backend_pdf import PdfPages

## Compute SOAP

In [ ]:
n_max = 8 ## Maximum number of radial basis function
l_max = None
cutoff = 4.0 ## Cutoff radiaus
sigma = 0.3 ## Broadening parameter of Gaussian 

center_atom_number = 7

# Loading the trajectory(XYZ File)
frames = aseio.read("../soap/osdb_project_summer/tpa.xyz", ":")

# hyperparameters for the SOAP descriptors
hypers = {
    "cutoff": cutoff,                                        # cutoff radius (Angstrom)
    "max_radial": n_max,                                      # number of radial basis functions
    "atomic_gaussian_width": sigma,                         # width of gaussian atomic density functions
    "radial_basis": {"Gto": {}},                          # type of radial functions
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}}, # type of cutoff function (damping features along radius)
    "center_atom_weight": 0.0,                            # center atom weight
}

#calculator = rascaline.SoapPowerSpectrum(**hypers) # Generate object - Power
#names_lis = ['center_type', 'neighbor_1_type', 'neighbor_2_type'] 

## Below is for radial Series ##
calculator = rascaline.SoapRadialSpectrum(**hypers) # Generate object - Radial
names_lis = ['center_type', 'neighbor_type']

## Specifying the [Center, Neighbor1, Neighbor2]

combi_knock = [[center_atom_number,6]]#(Must be 2-D array)

labels = Labels(names = names_lis, values = np.array(combi_knock, dtype = np.int32))

#Labels(['center_type'], np.asarray([[center_atom_number]], dtype=np.int32)) # Defining the the species center

In [ ]:
rho2i = calculator.compute(frames,selected_keys=labels)
print(rho2i)
rho2i_i = rho2i.keys_to_properties(['neighbor_type'])
print(rho2i_i)
soaps = rho2i_i.blocks(center_type=center_atom_number)[0].values # center_type choose the center species for local environment ( We select the C-centered SOAP )

# Number of centers (environments), number of SOAP features
print(soaps[0].shape)

## Compute real-space SOAP expansion

In [ ]:
# Number of species and unique species pairs in the molecule
n_species = 1

# Reshape the SOAP vector to have the shape
# (n_centers, n_species, n_max)
mol_soap = reshape_soaps(
    soaps[0],
    n_pairs=n_species,
    n_max=n_max,
    l_max=l_max
)

# Create real-space grids on which to expand the SOAP
# Radial grid
n_r_grid = int(1e+5)
r_grid = np.linspace(0.0, cutoff + 3 * sigma, n_r_grid)

# Compute real-space SOAP density with shape
# (n_centers, n_species, n_r_grid).
# This gives us the SOAP vectors as a function of r
mol_density = compute_soap_density(
    mol_soap,
    cutoff=cutoff,
    n_max=n_max,
    r_grid=r_grid,
    l_max=l_max,
    p_grid=None,
    chunk_size_r=-1,
    radial_basis='GTO',
    gaussian_sigma=sigma,
    projection_matrix=None
)

In [ ]:
np.savetxt("../../data_analysis/soap/osdb_project_summer/tpa_radial_density_N_C_8nmax.csv", \
           np.asarray(mol_density[0,0]), delimiter=',')

<span style="font-size:20px;font-weight:bold"> Plot SOAP density expansion </span>

In [ ]:
tma_mol = np.loadtxt("../../data_analysis/soap/osdb_project_summer/tma_radial_density_N_C.csv", delimiter=',')
tea_mol = np.loadtxt("../../data_analysis/soap/osdb_project_summer/tea_radial_density_N_C.csv", delimiter=',')
tpa_mol = np.loadtxt("../../data_analysis/soap/osdb_project_summer/tpa_radial_density_N_C.csv", delimiter=',')

fig, ax = plt.subplots(1, 1, figsize=(7, 6))

n_r_grid = int(1e+5)
r_grid = np.linspace(0.0, cutoff + 3 * sigma, n_r_grid)

ax.plot(r_grid, tma_mol, linewidth = 7, alpha = .7, label = "TMA")
ax.plot(r_grid, tea_mol , linewidth = 7, alpha = .7, label = "TEA")
ax.plot(r_grid, tpa_mol , linewidth = 7, alpha = .7, label = "TPA")

ax.set_title('N(Center) - C  Density', fontsize = 15)
ax.set_xlabel('r / Å', fontsize = 15)
ax.set_ylabel('$\\rho(r)$', fontsize = 15)

#ax.set_yticks([])
ax.axhline(y = 0.0, linestyle=':', linewidth=4, color = 'grey')
ax.legend(fontsize = 15, loc = 'best')

ax.xaxis.set_major_locator(MultipleLocator(1.0))
ax.xaxis.set_minor_locator(MultipleLocator(.2))

ax.tick_params(axis = 'both', direction = 'in', size = 7, width = 2.5,labelsize = 15, which = 'major')
ax.tick_params(axis = 'both', direction = 'in', size = 4.5, width = 1.5,labelsize = 15, which = 'minor')

pdf = PdfPages("../../data_analysis/soap/osdb_project_summer/N_C_radial_16nmax_nooffset.pdf")
fig.tight_layout()
pdf.savefig(fig)
pdf.close()
#plt.show()

## Real-space cumulative decision function

We'll create some fake data just to illustrate how to transform the SVM weights into real space.
(**NOTE:** This will only work if you use a linear SVM, we can't do this type of analysis with a nonlinear SVM)

If you are using scikit-learn's LinearSVC or SVC with `kernel='linear'`, you can get the weights as `svc.coef_`,
but you can also compute the weights as the dot product between the dual coefficients and the support vectors, i.e.,

`weights = svc.dual_coef_ @ svc.support_vectors_`

For a multi-class classification, you will have to do separate real-space expansions
for the weights corresponding to each individual binary classifier

(**NOTE:** If you centered and/or scaled your SOAP vectors before using them as input for the SVM,
you'll have to apply the same operations in real-space. For instance, if you mean-centered the SOAPs
and divided by some constant scale factor, in the SVM preprocessing step,
you need to convert the mean into a real-space form
---by passing it through the workflow just like you would for any other SOAP vector---
subtract the real-space mean from the real-space SOAP density, and then scale the real-space SOAP
density by the same scale factor from the SVM preprocessing routine)

In [ ]:
# We will just generate some random weights and intercept for a single binary classification as an example
seed = 100
rng = default_rng(seed)
weights = rng.random((1, soaps[0].shape[1])) # Weights have the shape (1, n_soap_features)
intercept = rng.random() * 1.0E-2

# Compute the real-space grid spacing
# (changes based on whether you are using a GTO or a DVR
# basis for your SOAP vectors); here we are using GTO
dr = np.diff(r_grid)[0] # dr for GTOs
# dr = 2.0 / len(r_grid) # dr for DVRs

# We reshape the weights just like we did for the SOAP vectors
mol_weights = reshape_soaps(
    weights,
    n_pairs=n_species,
    n_max=n_max,
    l_max=l_max
)

# We compute the weight density just like we did for the SOAP vectors.
# This gives us the weights as a function of r
mol_weight_density = compute_soap_density(
    mol_weights,
    cutoff=cutoff,
    n_max=n_max,
    r_grid=r_grid,
    l_max=l_max,
    p_grid=None,
    chunk_size_r=-1,
    radial_basis='GTO',
    gaussian_sigma=sigma,
    projection_matrix=None
)

Typically in SVM the decision function is the dot product between a feature vector (e.g., a SOAP vector) and the SVM weights, plus the intercept:

`decision_function = soap @ weights + intercept`

Now that we have both the SOAP vectors and the weights as a function of `r`, the decision function is instead:

`decision_function = dr * np.sum(real_space_soap * real_space_weights) + intercept`

To get the "spaghetti" line, we can replace `np.sum` with `np.cumsum` to get the value of the decision function for each `r`

In [ ]:
cumulative_decision_function = dr * np.cumsum(mol_density[center_atom_idx, species_idx] * mol_weight_density[0, species_idx]) + intercept

In [ ]:
# Add the spaghetti line to the plot we made before
fig = plt.figure(figsize=(3.5, 3.5))
ax = fig.add_subplot(1, 1, 1)
ax.plot(r_grid, mol_density[center_atom_idx, species_idx], label=label)
ax.set_xlabel('r')
ax.set_ylabel('Density')
ax.plot(r_grid, cumulative_decision_function, label='Decision function')
ax.legend()
plt.show()

You can in principle do this same thing with the three-body power spectrum,
but it is a bit more difficult. Below we compute and plot the real-space
power spectrum SOAP, but we will skip the real-space expansion of the SVM weights

# Power Spectrum

## Compute SOAP

In [ ]:
#####################
### SMILE -> XYZ ####
#####################

mol = Chem.rdmolfiles.MolFromSmiles("CC[N+](CC)(CC)CC")    # Constructing molecule from the SMILE,  Input the string
mol = Chem.AddHs(mol)                         # Adding H
AllChem.EmbedMolecule(mol)                    # Generate 3D coordinate

## Update needed for automate XYZ file writing

with open("../soap/osdb_project_summer/tma.xyz","w") as f:
    
  f.write(str(mol.GetNumAtoms()) + "\n\n")      # Total number of atoms
  #f.write(f"{15:.3f} {15:.3f} {15:.3f}\n")    # Lattice parameter (i.e. 10.0 Angstrom, Change needed)
  for atom in mol.GetAtoms():
      pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())                       # Gathering position of each atom -> pos
      f.write(str(atom.GetSymbol()) + f"    {pos.x:.4f}   {pos.y:.4f}   {pos.z:.4f}\n") # Writing into the XYZ format

In [ ]:
n_max = 12
l_max = 9
cutoff = 3.5
sigma = 0.3

center_atom_number = 7

# Loading the trajectory(XYZ File)
frames = aseio.read("../soap/osdb_project_summer/tma.xyz", ":")

# hyperparameters for the SOAP descriptors
hypers = {
    "cutoff": cutoff,                                        # cutoff radius (Angstrom)
    "max_radial": n_max,                                      # number of radial basis functions
    "max_angular": l_max,                                     # number of spherical harmonics (For three body correlation)
    "atomic_gaussian_width": sigma,                         # width of gaussian atomic density functions
    "radial_basis": {"Gto": {}},                          # type of radial functions
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}}, # type of cutoff function (damping features along radius)
    "center_atom_weight": 1.0,                            # center atom weight
}

calculator = rascaline.SoapPowerSpectrum(**hypers) # Generate object - Power
names_lis = ['center_type', 'neighbor_1_type', 'neighbor_2_type']

combi_knock = [[center_atom_number,6,6],\
               [center_atom_number,6,7],\
               [center_atom_number,7,7]] #(Must be 2-D array)

labels = Labels(names = names_lis, values = np.array(combi_knock, dtype = np.int32))

In [ ]:
rho2i = calculator.compute(frames,selected_keys=labels)
print(rho2i)

rho2i_i = rho2i.keys_to_properties(['neighbor_1_type', 'neighbor_2_type'])
print(rho2i_i)

soaps = rho2i_i.blocks(center_type=center_atom_number)[0].values # center_type choose the center species for local environment ( We select the C-centered SOAP )

## Compute real-space SOAP expansion

In [ ]:
# Number of species and unique species pairs in the molecule
#unique_species = np.unique(mol.get_atomic_numbers())
unique_species = np.asarray([6, 7])
unique_species_pairs = list(itertools.combinations_with_replacement(unique_species, 2))
print(unique_species_pairs)
n_species = len(unique_species)
n_species_pairs = len(unique_species_pairs)
# n_species_pairs = n_species * (n_species + 1) // 2

# Reshape the SOAP vector to have the shape
# (n_centers, n_pairs, n_max, n_max, l_max+1)
mol_soap = reshape_soaps(
    soaps[0],
    n_pairs=n_species_pairs,
    n_max=n_max,
    l_max=l_max
)

# Create real-space grids on which to expand the SOAP
# Radial grid
n_r_grid = 100
r_grid = np.linspace(0.0, cutoff + 3 * sigma, n_r_grid)

# Angular grid
n_p_grid = 100
p_grid = np.linspace(-1.0, 1.0, n_p_grid)

# Compute real-space SOAP density with shape
# (n_centers, n_species_pairs, n_r_grid, n_r'_grid, n_p_grid).
# This gives us the SOAP vectors as a function of r, r', and w
mol_density = compute_soap_density(
    mol_soap,
    cutoff=cutoff,
    n_max=n_max,
    r_grid=r_grid,
    l_max=l_max,
    p_grid=p_grid,
    chunk_size_r=-1,
    chunk_size_p=-1,
    radial_basis='GTO',
    gaussian_sigma=sigma,
    projection_matrix=None
)

## Plot SOAP density expansion

In [ ]:
# Function for plotting the real-space SOAP density with plotly
def make_plot(
    output, 
    r_grid, 
    p_grid, 
    density,
    species_pair_name=None,
    center_atom_name=None
):
    
    # Camera view settings
    x = -1.25
    y = -1.25
    z = 1.25

    # Compute aspect ratio from data
    # so we can set it manually including zoom
    # TODO: make this compatible with differing r_grid and p_grid sizes
    zoom = 1.0
    aspect_ratio_keys = ['x', 'y', 'z']
    xyz_max = np.amax(np.column_stack((r_grid, r_grid, p_grid)), axis=0)
    xyz_min = np.amin(np.column_stack((r_grid, r_grid, p_grid)), axis=0)
    xyz_ratios = np.abs(xyz_max - xyz_min)
    xyz_ratios = xyz_ratios / xyz_ratios[0] * zoom
    xyz_ratios[2] *= xyz_ratios[0] / xyz_ratios[2] # Make cube

    aspect_ratio = {key: value for key, value in zip(aspect_ratio_keys, xyz_ratios)}
        
    # Plot
    rx_grid, ry_grid, pz_grid = np.meshgrid(r_grid, r_grid, p_grid, indexing='ij')

    # Useful quantities for defining isosurface limits
    max = np.amax(density)
    min = np.amin(density)
    avg = np.mean(density)
    std = np.std(density)

    # Plot the isosurface
    fig = go.Figure()
    fig.add_trace(
        go.Isosurface(
            x=rx_grid.flatten(),
            y=ry_grid.flatten(),
            z= - pz_grid.flatten(),
            value= density.flatten(),
            coloraxis='coloraxis1',
            isomin=max - 12 * std,
            isomax=max - 6 * std,
            opacity=0.3,
            surface_count=4,
            caps=dict(x_show=False, y_show=False, z_show=False)
        )
    )

    # Plot title and layout
    font_size = 18
    species_pair_title = '-'.join(re.findall('[A-Z][a-z]*', species_pair_name))
    fig.update_layout(
        template='simple_white',
        title=dict(
            text=f'{center_atom_name}-<b>{species_pair_title}</b>',
            font=dict(size=2.0 * font_size),
            x=0.5, y=0.85,
            xanchor='center',
            yanchor='top'
        ),
        scene=dict(
            xaxis=dict(
                title=dict(text='r', font=dict(size=font_size)),
                tickfont=dict(size=0.75 * font_size),
                ticks='inside',
                tickwidth=2,
                linewidth=2,
                showgrid=True,
                mirror=True
            ),
            yaxis=dict(
                title=dict(text='r\'', font=dict(size=font_size)),
                tickfont=dict(size=0.75 * font_size),
                ticks='inside',
                tickwidth=2,
                linewidth=2,
                showgrid=True,
                mirror=True
            ),
            zaxis=dict(
                title=dict(text='w', font=dict(size=font_size)),
                tickfont=dict(size=0.75 * font_size),
                ticks='inside',
                tickwidth=2,
                linewidth=2,
                showgrid=True,
                mirror=True
            ),
            camera=dict(
                eye=dict(x=-1.25, y=-1.25, z=1.25),
                projection=dict(type='orthographic')
            ),
            aspectratio=aspect_ratio
        ),
        legend=dict(
            x=0.0, y=1.0,
            xanchor='left', yanchor='top',
            itemsizing='constant'
        ),
        coloraxis1=dict(
            colorscale='Reds',
            colorbar=dict(
                title=dict(
                    text='Density', 
                    side='right',
                    font=dict(size=font_size)
                ),
                ticks='inside',
                tickwidth=2,
                ticklen=10,
                tickfont=dict(size=font_size),
                outlinecolor='Black',
                outlinewidth=2,
                len=0.5,
                x=-0.02, xanchor='right'
            ),
        ),
        font=dict(size=14, family='helvetica, sans-serif'),
        autosize=False, width=800, height=800,
    )
    
    fig.write_html(f'{output}.html')
    return fig

In [ ]:
# Possible species pairings for describing SOAP environments
unique_species_pairs_symbols = [
    (chemical_symbols[z1], chemical_symbols[z2]) for (z1, z2) in unique_species_pairs
]

print(unique_species_pairs_symbols)

In [ ]:
# Examine the environment of the C atom (i.e., the atom with index 0)
center_atom_idx = 0

# Look at hydrogen-hydrogen correlations in the environment
# (i.e., the species pair with index 0)
species_pair_idx =0

# Plot the density correlations for the specified atom
# and for the specified atomic correlations.
# Writes a file 'molecule.html' with the density plot;
# open this file with your web browser to get an interactive plot
fig = make_plot(
    'tma_NCC', 
    r_grid, 
    p_grid, 
    mol_density[center_atom_idx, species_pair_idx],
    center_atom_name=chemical_symbols[center_atom_number],
    species_pair_name=''.join(unique_species_pairs_symbols[species_pair_idx]),
)

In [ ]:
fig.show() # We can show the density directly in the notebook, but it is very slow

# Function documentation

For reference

In [ ]:
help(librascal_soap)

In [ ]:
help(reshape_soaps)

In [ ]:
help(compute_soap_density)